In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import urllib.request
import json

In [2]:
def get_json(url):
    with urllib.request.urlopen(url) as response:
        return json.loads(response.read())

In [ ]:
#Fetch cholesterol + gene
genelist = pd.read_csv("DistFeatures.csv")[["GeneName", "progression_arch Average", "regression_arch Average", "progression_arch Log2 Fold Change", "progression_arch P-Value"]]

count_apiurl = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=0&sort=relevance&term='

querylist = [
    {"title": "Wikipedia", "url": "https://en.wikipedia.org/w/index.php?search=", "pubmed": 0},
    {"title": "Pubmed: Cholesterol", "url": "https://www.ncbi.nlm.nih.gov/pubmed/?term=", "term": "cholesterol+", "pubmed": 1},
    {"title": "Pubmed: Cholesterol+Macrophages", "url": "https://www.ncbi.nlm.nih.gov/pubmed/?term=", "term": "cholesterol+macrophage+", "pubmed": 1},
]
genefetch = pd.DataFrame(genelist)
for entry in querylist:
    genefetch[entry["title"]] = genefetch["GeneName"].apply(lambda x : '<a href="{}" target="_blank">{}</a>'.format(
        entry["url"]+entry.get("term", "")+x,
        entry["title"]+"+"+x+("<br>(n="+get_json(count_apiurl+entry.get("term", "")+"+"+x)["esearchresult"]["count"]+")" if entry["pubmed"] else "") 
    ))
genefetch.to_html("GeneReport.html", escape=0)
genefetch.style.format(lambda x: x)